In [10]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
digits = datasets.load_digits() # 데이터셋 로드
features = digits.data # 특성 행렬을 만듭니다.
target = digits.target # 타깃 벡터를 만듭니다.
standardizer = StandardScaler() # 표준화 객체를 만듭니다.
logit = LogisticRegression() # # 로지스틱 회귀 객체를 만듭니다

# 표준화한 다음 로지스틱 회귀를 실행하는 파이프라인을 만듭니다.
pipeline = make_pipeline(standardizer, logit)
kf = KFold(n_splits=10, shuffle=True, random_state=1) # k-폴드 교차검증을 만듭니다. (10 개의 폴드를 만듬)

# k-폴드 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline, features, target, cv=kf, # 교차 검증 기법
                            scoring="accuracy", # 평가 지표
                            n_jobs=-1) # 모든 CPU 코어 사용
cv_results.mean() # 평균을 계산


0.964931719428926

In [11]:
cv_results ## 10개 폴드의 점수를 모두 확인(평가 점수는 cv_results에 저장)

array([0.97222222, 0.97777778, 0.95555556, 0.95      , 0.95555556,
       0.98333333, 0.97777778, 0.96648045, 0.96089385, 0.94972067])

In [2]:
from sklearn.model_selection import train_test_split
# 훈련 세트와 테스트 세트를 만듭니다.
features_train, features_test, target_train, target_test = train_test_split( features, target, test_size=0.1, random_state=1)
standardizer.fit(features_train) # 훈련 세트로 standardizer의 fit 메서드를 호출

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
# 훈련 세트와 테스트 세트에 모두 적용합니다.
features_train_std = standardizer.transform(features_train)
features_test_std = standardizer.transform(features_test)

In [8]:
#훈련 폴드로 50%, 테스트 폴드로 20%를 사용하여 10번 반복하는 실습입니다.
from sklearn.model_selection import ShuffleSplit
# ShuffleSplit 분할기를 만듭니다.
ss = ShuffleSplit(n_splits=10, train_size=0.5, test_size=0.2, random_state=42)
# 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline, # 파이프라인
                            features, # 특성 행렬
                            target, # 타깃 벡터
                            cv=ss, # 교차 검증 기법
                            scoring="accuracy", # 평가 지표
                            n_jobs=-1) # 모든 CPU 코어 사용
cv_results.mean() # 평균을 계산합니다.

0.95

In [13]:
# 사이킷런에서 교차 검증을 반복하여 실행할 수 있는 RepeatedKFold 와 StratifiedRepeatedKFold
# n_splits 매개변수 기본값은 5이고 n_repeats 매개변수 기본값은 10입니다.

from sklearn.model_selection import RepeatedKFold

# RepeatedKFold 분할기를 만듭니다.
rfk = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
# 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline, # 파이프라인
                            features, # 특성 행렬
                            target, # 타깃 벡터
                            cv=rfk, # 교차 검증 기법
                            scoring="accuracy", # 평가 지표
                            n_jobs=-1) # 모든 CPU 코어 사용
len(cv_results) # 검증 점수 개수를 확인

50

In [14]:
cv_results.mean()

0.9629416511483552

## 모델 평가

### 선형회귀

#### 교차 특성 처리

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
boston = load_boston() # 데이터를 로드하고 두 개의 특성만 선택
features = boston.data[:,0:2]
target = boston.target
# 교차 항을 만듭니다.
interaction = PolynomialFeatures( degree=3, include_bias=False, interaction_only=True)
features_interaction = interaction.fit_transform(features)
regression = LinearRegression() # 선형 회귀 모델 객체 생성
model = regression.fit(features_interaction, target) # 선형 회귀 모델 훈련
features[0] # 첫 번째 샘플의 특성 값을 확인
import numpy as np
# 각 샘플에서 첫 번째와 두 번째 특성을 곱합니다.
interaction_term = np.multiply(features[:, 0], features[:, 1])
interaction_term[0] # 첫 번째 샘플의 교차 항을 확인.
features_interaction[0] # 첫 번째 샘플의 값을 확인

array([6.3200e-03, 1.8000e+01, 1.1376e-01])

#### 비선형 관계 학습

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
boston = load_boston() # 데이터를 로드하고 하나의 특성을 선택
features = boston.data[:,0:1]
target = boston.target
# 다항 특성 x^2와 x^3를 만듭니다.
polynomial = PolynomialFeatures(degree=3, include_bias=False)
features_polynomial = polynomial.fit_transform(features)
regression = LinearRegression() # 선형 회귀 모델 객체 생성
model = regression.fit(features_polynomial, target) # 선형 회귀 모델 훈련
features[0] # 첫 번째 샘플을 확인
features[0]**2 # 첫 번째 샘플을 x^2로 거듭제곱합니다.
features[0]**3 # 첫 번째 샘플을 x^3로 세제곱합니다.
features_polynomial[0] # 첫 번째 샘플의 x, x^2,x^3 값을 확인

array([6.32000000e-03, 3.99424000e-05, 2.52435968e-07])

In [3]:
from sklearn.linear_model import Ridge
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
boston = load_boston() # 데이터 로드
features = boston.data
target = boston.target
scaler = StandardScaler() # 특성을 표준화
features_standardized = scaler.fit_transform(features)
regression = Ridge(alpha=0.5) # alpha 값을 지정한 릿지 회귀를 만듭니다.
model = regression.fit(features_standardized, target) # 선형 회귀 모델을 훈련합니다.

In [5]:
from sklearn.linear_model import RidgeCV
regr_cv = RidgeCV(alphas=[0.1, 1.0, 10.0]) # 세 개의 alpha 값에 대한 릿지 회귀 객체 생성
model_cv = regr_cv.fit(features_standardized, target) # 선형 회귀 모델 훈련
model_cv.coef_ # 계수 확인

array([-0.91987132,  1.06646104,  0.11738487,  0.68512693, -2.02901013,
        2.68275376,  0.01315848, -3.07733968,  2.59153764, -2.0105579 ,
       -2.05238455,  0.84884839, -3.73066646])

In [6]:
model_cv.alpha_ 

1.0

In [7]:
regr_cv = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=5) # 5-폴드 교차검증을 사용하여 릿지 회귀 객체 생성
model_cv = regr_cv.fit(features_standardized, target) # 선형 회귀 모델을 훈련합니다.
model_cv.alpha_ # alpha 값을 확인

C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


10.0

In [8]:
regr_cv = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=5) # 5-폴드 교차검증을 사용하여 릿지 회귀 객체 생성

In [9]:
model_cv = regr_cv.fit(features_standardized, target) # 선형 회귀 모델을 훈련
model_cv.alpha_ # alpha 값을 확인

C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


10.0

### 라쏘회귀

In [10]:
from sklearn.linear_model import Lasso
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
boston = load_boston() # 데이터 로드
features = boston.data
target = boston.target
scaler = StandardScaler() # 특성 표준화
features_standardized = scaler.fit_transform(features)
regression = Lasso(alpha=0.5) # alpha 값을 지정한 라쏘 회귀 객체 생성  (비율)
model = regression.fit(features_standardized, target) # 선형 회귀 모델 훈련
model.coef_ # 계수 확인
regression_a10 = Lasso(alpha=10) # 큰 alpha 값을 지정한 라쏘 회귀 객체 생성  (특정갯수)
model_a10 = regression_a10.fit(features_standardized, target)
model_a10.coef_ 

array([-0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.])

In [12]:
from sklearn.linear_model import LassoCV
# 세 개의 alpha 값에 대한 라쏘 회귀를 만듭니다.
lasso_cv = LassoCV(alphas=[0.1, 1.0, 10.0], cv=5)
model_cv = lasso_cv.fit(features_standardized, target) # 선형 회귀 모델 훈련
model_cv.coef_ # 계수를 확인

array([-0.63230364,  0.70840931, -0.        ,  0.65760723, -1.57419335,
        2.82626903, -0.        , -2.42207901,  1.19593681, -0.84646778,
       -1.92249345,  0.76216539, -3.72618383])

In [13]:
model_cv.alpha_ # alpha 값을 확인

0.1

In [14]:
sns# 1000개의 alpha 값을 탐색하는 라쏘 회귀를 만듭니다.
lasso_cv = LassoCV(n_alphas=1000, cv=5)
model_cv = lasso_cv.fit(features_standardized, target) # 선형 회귀 모델 훈련
model_cv.alpha_ # 계수를 확인sns

0.15326173083090813

In [15]:
lasso_cv.alphas_ # alpha 값을 확인

array([6.77765364e+00, 6.73095006e+00, 6.68456831e+00, 6.63850616e+00,
       6.59276142e+00, 6.54733189e+00, 6.50221542e+00, 6.45740983e+00,
       6.41291299e+00, 6.36872277e+00, 6.32483706e+00, 6.28125375e+00,
       6.23797077e+00, 6.19498605e+00, 6.15229752e+00, 6.10990315e+00,
       6.06780092e+00, 6.02598880e+00, 5.98446481e+00, 5.94322695e+00,
       5.90227325e+00, 5.86160175e+00, 5.82121052e+00, 5.78109761e+00,
       5.74126112e+00, 5.70169913e+00, 5.66240975e+00, 5.62339112e+00,
       5.58464135e+00, 5.54615860e+00, 5.50794102e+00, 5.46998680e+00,
       5.43229411e+00, 5.39486116e+00, 5.35768615e+00, 5.32076731e+00,
       5.28410287e+00, 5.24769107e+00, 5.21153018e+00, 5.17561847e+00,
       5.13995423e+00, 5.10453573e+00, 5.06936130e+00, 5.03442925e+00,
       4.99973792e+00, 4.96528563e+00, 4.93107075e+00, 4.89709163e+00,
       4.86334666e+00, 4.82983422e+00, 4.79655271e+00, 4.76350054e+00,
       4.73067612e+00, 4.69807789e+00, 4.66570428e+00, 4.63355376e+00,
      